**NOTEBOOK 3: FINE-TUNING PHI-2 WITH LORA**


Task 3: Fine-tuning Phi-2 for Text Summarization with XSum
UAS Deep Learning - Final Term Assignment

This notebook:
1. Loads formatted data from Google Drive
2. Configures Phi-2 model with LoRA for efficient fine-tuning
3. Sets up training with optimal hyperparameters
4. Trains the model with monitoring
5. Saves the fine-tuned model

**SETUP & INSTALLATIONS**

In [ ]:
print("="*70)
print("NOTEBOOK 3: FINE-TUNING PHI-2 WITH LORA")
print("="*70)

# Install required libraries
print("\nInstalling libraries (this may take a few minutes)...")
!pip install 'datasets==3.1.0' -q
!pip install transformers accelerate -q
!pip install peft -q  # For LoRA
!pip install bitsandbytes -q  # For 4-bit quantization (QLoRA)
!pip install trl -q  # For SFTTrainer

print("✓ Installation complete!")

# Imports
import torch
import json
from datasets import load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import os

print("✓ Libraries imported successfully!")

# Check GPU
print("\n" + "="*70)
print("GPU CONFIGURATION")
print("="*70)
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"CUDA Version: {torch.version.cuda}")
else:
    print("⚠️  WARNING: No GPU detected! Training will be very slow.")
    print("   Go to Runtime > Change runtime type > Select GPU (T4)")

NOTEBOOK 3: FINE-TUNING PHI-2 WITH LORA

Installing libraries (this may take a few minutes)...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.9 MB/s eta 0:00:00


**LOAD PREPROCESSED DATA FROM DRIVE**

In [ ]:
print("\n" + "="*70)
print("LOADING DATA FROM GOOGLE DRIVE")
print("="*70)

# Mount Google Drive
drive.mount('/content/drive')
PROJECT_DIR = '/content/drive/MyDrive/DL_FinalTask_XSum'

print(f"✓ Mounted Drive at: {PROJECT_DIR}")

# Load formatting config
with open(f'{PROJECT_DIR}/formatting_config.json', 'r') as f:
    formatting_config = json.load(f)

print("\n✓ Loaded formatting configuration")

# Choose dataset size
# For initial experiments: use small dataset (faster)
# For final training: use full dataset
USE_SMALL_DATASET = True  # Change to False for full training

if USE_SMALL_DATASET:
    print("\n📊 Loading SMALL dataset (for fast experimentation)...")
    tokenized_datasets = load_from_disk(f'{PROJECT_DIR}/xsum_tokenized_small')
else:
    print("\n📊 Loading FULL dataset (for final training)...")
    tokenized_datasets = load_from_disk(f'{PROJECT_DIR}/xsum_tokenized_full')

# Remove the 'labels' column to let DataCollatorForLanguageModeling generate them
# This is a common fix for ValueError when pre-existing labels conflict with collator's padding logic
if 'labels' in tokenized_datasets['train'].features:
    tokenized_datasets = tokenized_datasets.remove_columns(['labels'])
    print("✓ Removed 'labels' column from dataset to ensure proper collator behavior.")

print(f"✓ Dataset loaded!")
print(f"   Training samples: {len(tokenized_datasets['train']):,}")
print(f"   Validation samples: {len(tokenized_datasets['validation']):,}")

**LOAD PHI-2 MODEL WITH QUANTIZATION**

In [ ]:
print("\n" + "="*70)
print("LOADING PHI-2 MODEL")
print("="*70)

# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/phi-2",
    trust_remote_code=True
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print("✓ Tokenizer loaded")

# Configure 4-bit quantization for memory efficiency (QLoRA)
print("\nConfiguring 4-bit quantization (QLoRA)...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print("✓ Quantization config created")

# Load model with quantization
print("\nLoading Phi-2 model (this may take 2-3 minutes)...")
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

print("✓ Model loaded successfully!")
print(f"   Model device: {model.device}")
print(f"   Model dtype: {model.dtype}")

# Get model size
param_count = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"   Total parameters: {param_count:,}")
print(f"   Trainable parameters: {trainable_params:,}")

**CONFIGURE LORA**

In [ ]:
print("\n" + "="*70)
print("CONFIGURING LORA (LOW-RANK ADAPTATION)")
print("="*70)

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)
print("✓ Model prepared for k-bit training")

# LoRA configuration
lora_config = LoraConfig(
    r=16,  # Rank - higher = more parameters but better learning
    lora_alpha=32,  # Scaling factor
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "dense",
        "fc1",
        "fc2"
    ],  # Which layers to apply LoRA
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

print("LoRA Configuration:")
print(f"   Rank (r): {lora_config.r}")
print(f"   Alpha: {lora_config.lora_alpha}")
print(f"   Dropout: {lora_config.lora_dropout}")
print(f"   Target modules: {lora_config.target_modules}")

# Apply LoRA to model
model = get_peft_model(model, lora_config)
print("\n✓ LoRA applied to model!")

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
trainable_percent = 100 * trainable_params / all_params

print(f"\n📊 Training Statistics:")
print(f"   Total parameters: {all_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Trainable %: {trainable_percent:.4f}%")
print(f"   Memory savings: ~{100 - trainable_percent:.2f}% reduction in training params!")

**FIX DATASET FORMAT**

In [ ]:
print("\n" + "="*70)
print("FIXING DATASET FORMAT")
print("="*70)

def fix_labels(example):
    """Ensure labels are properly formatted as a list"""
    # If labels don't exist or are None, copy from input_ids
    if 'labels' not in example or example['labels'] is None:
        example['labels'] = example['input_ids'][:]

    # Ensure both are lists of ints
    if not isinstance(example['input_ids'], list):
        example['input_ids'] = list(example['input_ids'])
    if not isinstance(example['labels'], list):
        example['labels'] = list(example['labels'])

    # Ensure they're the same length
    if len(example['input_ids']) != len(example['labels']):
        example['labels'] = example['input_ids'][:]

    return example

print("Applying format fixes to dataset...")
tokenized_datasets = tokenized_datasets.map(
    fix_labels,
    desc="Fixing labels"
)

# Remove any columns that might cause issues
columns_to_keep = ['input_ids', 'attention_mask', 'labels']
columns_to_remove = [col for col in tokenized_datasets['train'].column_names
                     if col not in columns_to_keep]

if columns_to_remove:
    print(f"Removing extra columns: {columns_to_remove}")
    tokenized_datasets = tokenized_datasets.remove_columns(columns_to_remove)

print(f"✓ Dataset fixed!")
print(f"   Columns: {tokenized_datasets['train'].column_names}")

# Verify fix worked
sample = tokenized_datasets['train'][0]
print(f"\n✓ Verification:")
print(f"   input_ids: list of {len(sample['input_ids'])} ints")
print(f"   labels: list of {len(sample['labels'])} ints")
print(f"   Match: {len(sample['input_ids']) == len(sample['labels'])}")

**SETUP DATA COLLATOR**

In [ ]:
print("\n" + "="*70)
print("SETTING UP DATA COLLATOR")
print("="*70)

from transformers import DataCollatorForSeq2Seq

# Use DataCollatorForSeq2Seq which handles padding better
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100,
    padding=True
)

print("✓ Data collator configured")
print("   Type: Seq2Seq with dynamic padding")
print("   Label padding: -100 (ignored in loss)")

**CONFIGURE TRAINING ARGUMENTS**

In [ ]:
print("\n" + "="*70)
print("CONFIGURING TRAINING HYPERPARAMETERS")
print("="*70)

OUTPUT_DIR = "/content/phi2-xsum-lora"
os.makedirs(OUTPUT_DIR, exist_ok=True)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    optim="paged_adamw_8bit",
    weight_decay=0.01,
    max_grad_norm=1.0,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    logging_steps=25,
    logging_dir=f"{OUTPUT_DIR}/logs",
    report_to="none",
    fp16=True,
    dataloader_num_workers=0,  # IMPORTANT: Set to 0 to avoid multiprocessing issues
    remove_unused_columns=False,  # Keep all columns for now
    push_to_hub=False,
    seed=42
)

print("✓ Training configured")

**INITIALIZE TRAINER**

In [ ]:
print("\n" + "="*70)
print("INITIALIZING TRAINER")
print("="*70)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

print("✓ Trainer initialized!")

# CRITICAL: Test that batching works before training
print("\n🔍 Testing data loading...")
try:
    test_dataloader = trainer.get_train_dataloader()
    test_batch = next(iter(test_dataloader))
    print(f"✓ Data loading successful!")
    print(f"   Batch size: {test_batch['input_ids'].shape}")
    print(f"   Labels shape: {test_batch['labels'].shape}")
    del test_dataloader, test_batch
except Exception as e:
    print(f"❌ Data loading failed: {e}")
    print("\nDebugging info:")
    sample = tokenized_datasets['train'][0]
    for key in sample.keys():
        print(f"   {key}: {type(sample[key])}, length: {len(sample[key]) if hasattr(sample[key], '__len__') else 'N/A'}")
    raise

In [ ]:
# ============================================================================
# SECTION 7B: ENHANCED PROGRESS TRACKING WITH LIVE PLOTTING
# ============================================================================

print("\n" + "="*70)
print("SETTING UP ENHANCED PROGRESS TRACKING")
print("="*70)

from transformers import TrainerCallback
from tqdm.auto import tqdm
import time
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

class EnhancedProgressCallback(TrainerCallback):
    """Callback with progress bar, ETA, and live loss plotting"""

    def __init__(self):
        self.training_bar = None
        self.start_time = None
        self.losses = []
        self.steps = []
        self.eval_losses = []
        self.eval_steps = []

    def on_train_begin(self, args, state, control, **kwargs):
        self.start_time = time.time()
        total_steps = state.max_steps
        print(f"\n🚀 Training Started!")
        print(f"{'='*70}")
        print(f"📊 Configuration:")
        print(f"   Total steps: {total_steps}")
        print(f"   Total epochs: {args.num_train_epochs}")
        print(f"   Train samples: {len(trainer.train_dataset):,}")
        print(f"   Batch size: {args.per_device_train_batch_size}")
        print(f"   Gradient accumulation: {args.gradient_accumulation_steps}")
        print(f"   Effective batch size: {args.per_device_train_batch_size * args.gradient_accumulation_steps}")
        print(f"   Learning rate: {args.learning_rate}")
        print(f"{'='*70}\n")

        self.training_bar = tqdm(
            total=total_steps,
            desc="🔥 Training",
            bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]'
        )

    def on_log(self, args, state, control, logs=None, **kwargs):
        """Called when logging happens"""
        if logs:
            if 'loss' in logs:
                self.losses.append(logs['loss'])
                self.steps.append(state.global_step)
            if 'eval_loss' in logs:
                self.eval_losses.append(logs['eval_loss'])
                self.eval_steps.append(state.global_step)

    def on_step_end(self, args, state, control, **kwargs):
        if self.training_bar:
            self.training_bar.update(1)

            # Calculate time estimates
            elapsed = time.time() - self.start_time
            steps_done = state.global_step
            steps_remaining = state.max_steps - steps_done

            if steps_done > 0:
                time_per_step = elapsed / steps_done
                eta_seconds = time_per_step * steps_remaining

                # Format times
                elapsed_str = self._format_time(elapsed)
                eta_str = self._format_time(eta_seconds)

                # Get current metrics
                current_loss = self.losses[-1] if self.losses else 0
                current_lr = state.log_history[-1].get('learning_rate', 0) if state.log_history else 0

                # Update progress bar
                self.training_bar.set_postfix({
                    'loss': f'{current_loss:.4f}',
                    'lr': f'{current_lr:.2e}',
                    'elapsed': elapsed_str,
                    'ETA': eta_str
                })

    def on_evaluate(self, args, state, control, **kwargs):
        """Show quick evaluation update"""
        if self.eval_losses:
            print(f"\n📊 Evaluation at step {state.global_step}: loss = {self.eval_losses[-1]:.4f}")

    def on_train_end(self, args, state, control, **kwargs):
        if self.training_bar:
            self.training_bar.close()

        total_time = time.time() - self.start_time
        print(f"\n{'='*70}")
        print(f"✅ TRAINING COMPLETED!")
        print(f"{'='*70}")
        print(f"⏱️  Total time: {self._format_time(total_time)}")
        print(f"📉 Final train loss: {self.losses[-1]:.4f}")
        if self.eval_losses:
            print(f"📉 Final eval loss: {self.eval_losses[-1]:.4f}")
        print(f"⚡ Average speed: {state.max_steps/total_time:.2f} steps/sec")

        # Plot training curve
        self._plot_training_curve()

    def _format_time(self, seconds):
        """Format seconds into human readable time"""
        if seconds < 60:
            return f"{seconds:.0f}s"
        elif seconds < 3600:
            return f"{seconds/60:.1f}min"
        else:
            hours = seconds / 3600
            return f"{hours:.1f}h"

    def _plot_training_curve(self):
        """Plot training and validation loss curves"""
        if not self.losses:
            return

        plt.figure(figsize=(12, 5))

        # Training loss
        plt.subplot(1, 2, 1)
        plt.plot(self.steps, self.losses, label='Training Loss', color='blue', alpha=0.7)
        if self.eval_losses:
            plt.scatter(self.eval_steps, self.eval_losses,
                       label='Validation Loss', color='red', s=50, zorder=5)
        plt.xlabel('Steps')
        plt.ylabel('Loss')
        plt.title('Training Progress')
        plt.legend()
        plt.grid(alpha=0.3)

        # Loss moving average
        plt.subplot(1, 2, 2)
        if len(self.losses) > 10:
            window = min(50, len(self.losses) // 10)
            smoothed = np.convolve(self.losses, np.ones(window)/window, mode='valid')
            smoothed_steps = self.steps[window-1:]
            plt.plot(smoothed_steps, smoothed, label='Smoothed Training Loss',
                    color='blue', linewidth=2)
        else:
            plt.plot(self.steps, self.losses, label='Training Loss', color='blue')

        if self.eval_losses:
            plt.scatter(self.eval_steps, self.eval_losses,
                       label='Validation Loss', color='red', s=50, zorder=5)
        plt.xlabel('Steps')
        plt.ylabel('Loss')
        plt.title('Smoothed Training Progress')
        plt.legend()
        plt.grid(alpha=0.3)

        plt.tight_layout()
        plt.savefig(f'{OUTPUT_DIR}/training_curve.png', dpi=300, bbox_inches='tight')
        print(f"\n✓ Training curve saved to: {OUTPUT_DIR}/training_curve.png")
        plt.show()

# Add callback
progress_callback = EnhancedProgressCallback()
trainer.add_callback(progress_callback)

print("✓ Enhanced progress tracking enabled!")
print("   Features:")
print("   ✓ Real-time progress bar with ETA")
print("   ✓ Current loss and learning rate display")
print("   ✓ Elapsed time tracking")
print("   ✓ Training curve visualization at end")


**START TRAINING**

In [ ]:
print("\n" + "="*70)
print("🚀 STARTING TRAINING")
print("="*70)

training_result = trainer.train()

print("\n✅ TRAINING COMPLETE!")

**EVALUATE MODEL**

In [ ]:
print("\n" + "="*70)
print("EVALUATING TRAINED MODEL")
print("="*70)

eval_results = trainer.evaluate()

print("\n📊 Evaluation Results:")
print(f"   Eval loss: {eval_results['eval_loss']:.4f}")
print(f"   Eval runtime: {eval_results['eval_runtime']:.2f} seconds")
print(f"   Eval samples/second: {eval_results['eval_samples_per_second']:.2f}")
print(f"   Perplexity: {np.exp(eval_results['eval_loss']):.2f}")

**SAVE MODEL TO DRIVE**

In [ ]:
print("\n" + "="*70)
print("SAVING FINE-TUNED MODEL")
print("="*70)

# Save locally first
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"✓ Model saved locally to: {OUTPUT_DIR}")

# Copy to Google Drive
DRIVE_MODEL_DIR = f"{PROJECT_DIR}/phi2_finetuned_model"
print(f"\nCopying model to Google Drive: {DRIVE_MODEL_DIR}")

import shutil
if os.path.exists(DRIVE_MODEL_DIR):
    shutil.rmtree(DRIVE_MODEL_DIR)
shutil.copytree(OUTPUT_DIR, DRIVE_MODEL_DIR)

print("✓ Model saved to Google Drive!")

# Save training metrics
metrics = {
    'training': {
        'train_loss': training_result.metrics['train_loss'],
        'train_runtime': training_result.metrics['train_runtime'],
        'train_samples_per_second': training_result.metrics['train_samples_per_second'],
    },
    'evaluation': {
        'eval_loss': eval_results['eval_loss'],
        'perplexity': float(np.exp(eval_results['eval_loss'])),
    },
    'configuration': {
        'num_epochs': training_args.num_train_epochs,
        'batch_size': training_args.per_device_train_batch_size,
        'learning_rate': training_args.learning_rate,
        'lora_r': lora_config.r,
        'lora_alpha': lora_config.lora_alpha,
        'dataset_size': 'small' if USE_SMALL_DATASET else 'full',
        'train_samples': len(tokenized_datasets['train']),
        'val_samples': len(tokenized_datasets['validation'])
    }
}

with open(f'{DRIVE_MODEL_DIR}/training_metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("✓ Training metrics saved!")

**TEST GENERATION**

In [ ]:
print("\n" + "="*70)
print("TESTING MODEL GENERATION")
print("="*70)

# Test on a few validation examples
print("\nGenerating summaries for sample articles...\n")

# Get test examples
test_examples = tokenized_datasets['validation'].select(range(3))

for i, example in enumerate(test_examples):
    print(f"{'='*70}")
    print(f"Example {i+1}")
    print(f"{'='*70}")

    # Decode the input (will include the prompt)
    input_text = tokenizer.decode(example['input_ids'][:800], skip_special_tokens=True)

    # Extract just the article part (before "Summarize")
    if "Summarize the above article" in input_text:
        article_part = input_text.split("Summarize the above article")[0].replace("Article:", "").strip()
    else:
        article_part = input_text[:300]

    print(f"\nArticle (first 300 chars):\n{article_part[:300]}...")

    # Ground truth
    print(f"\nGround Truth Summary:\n{example['target_summary']}")

    # Generate with the model
    prompt = f"Article: {article_part}\n\nSummarize the above article in one sentence.\nSummary:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            num_beams=4,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract just the summary part (after "Summary:")
    if "Summary:" in generated_text:
        generated_summary = generated_text.split("Summary:")[-1].strip()
    else:
        generated_summary = generated_text

    print(f"\nGenerated Summary:\n{generated_summary}")
    print()

print("="*70)
print("✅ MODEL TESTING COMPLETE!")
print("="*70)

**FINAL SUMMARY**

In [ ]:
print("\n" + "="*70)
print("🎉 NOTEBOOK 3 COMPLETE!")
print("="*70)

print("\n✅ Achievements:")
print("   1. Loaded and prepared Phi-2 model with 4-bit quantization")
print("   2. Applied LoRA for efficient fine-tuning")
print(f"   3. Trained for {training_args.num_train_epochs} epochs")
print(f"   4. Achieved eval loss: {eval_results['eval_loss']:.4f}")
print(f"   5. Saved fine-tuned model to Google Drive")
print("   6. Tested generation on sample articles")

print(f"\n📁 Model Location:")
print(f"   {DRIVE_MODEL_DIR}")

print("\n🚀 Next Steps:")
print("   → Proceed to Notebook 4: Comprehensive Evaluation & Analysis")
print("   → Calculate ROUGE scores")
print("   → Generate analysis visualizations")
print("   → Create final report")